In [2]:
import pandas as pd
import numpy as np

In [1]:
data_dir = 'data/'

In [3]:
bert_rank_df = pd.read_csv(data_dir + 'bert_thesis_dataset_top100.tsv',delimiter='\t',encoding='utf-8',header=None)
bert_rank_df.columns = ['query_id', 'passage_id','bm25_rank','query','passage','bert_score', 'bert_rank']
og_qrels_df = pd.read_csv(data_dir + 'qrels.dev.small.tsv',delimiter='\t',encoding='utf-8',header=None)
og_qrels_df.columns = ['query_id','label1','passage_id','label2']
queries_df = pd.read_csv(data_dir + 'queries.dev.small.tsv',delimiter='\t',encoding='utf-8',header=None)
queries_df.columns = ['query_id','query']
new_qrels_df = pd.read_csv(data_dir + 'thesis_qrels.tsv',delimiter='\t',encoding='utf-8',header=None)
new_qrels_df.columns = ['query_id','label1','passage_id','label2']

In [4]:
og_qrels_list = []
for index, row in og_qrels_df.iterrows():
    [query_id,passage_id] = row.values[::2]
    og_qrels_list.append((query_id,passage_id))

new_qrels_list = []
for index, row in new_qrels_df.iterrows():
    [query_id,passage_id] = row.values[::2]
    new_qrels_list.append((query_id,passage_id))

In [5]:
og_relevance_column_list = []
new_relevance_column_list = []
for index, row in bert_rank_df.iterrows():
    [query_id,passage_id] = row.values[:2]
    if (query_id,passage_id) in og_qrels_list:
        og_relevance_column_list.append(1)
    else:
        og_relevance_column_list.append(0)
    if (query_id,passage_id) in new_qrels_list:
        new_relevance_column_list.append(1)
    else:
        new_relevance_column_list.append(0)

In [6]:
bert_rank_df['og_rel'] = og_relevance_column_list
bert_rank_df['new_rel'] = new_relevance_column_list

In [7]:
len(bert_rank_df[(bert_rank_df['bert_rank'] == 1) & (bert_rank_df['og_rel'] == 0) & (bert_rank_df['new_rel'] == 1)])

12

12 cases where the original msmarco relevant passage is not ranked 1 by BERT but a newly labeled relevant passage is.

In [8]:
len(bert_rank_df[(bert_rank_df['bert_rank'] == 1) & (bert_rank_df['og_rel'] == 0) & (bert_rank_df['new_rel'] == 0)])

7

7 cases where both the original msmarco relevant passage and the newly ranked relevant passage are not ranked 1

In [9]:
len(bert_rank_df[(bert_rank_df['bert_rank'] == 1) & (bert_rank_df['og_rel'] == 1) & (bert_rank_df['new_rel'] == 1)])

23

23 cases where both the original msmarco relevant passage and newly ranked relevant passage are ranked 1.

In [10]:
len(bert_rank_df[(bert_rank_df['bert_rank'] == 1) & (bert_rank_df['og_rel'] == 1) & (bert_rank_df['new_rel'] == 0)])

0

0 cases where the msmarco relevant passage is not ranked relevant.